<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/two_by_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Two-by-Two Encoding

This notebook shows that every 1# program may be modified to get a program with the special feature that when it runs, the case statement 1##### is never called on an empty R1.  (This technical feature will be put to use later, when we prove the undecidability of Post's Correspondence Problem).

The work here also is a good illustration of the use of the Sanity tool.

In [ ]:
#@title
!python -m pip install -U setuptools
!python -m pip install -U git+https://github.com/lmoss/onesharp/
from onesharp.interpreter.interpreter import *
from onesharp.tools.sanity import *

The two-by-two encoding takes the symbol 1 and replaces it by 11, and the symbol # and replaces it by 1#.  Then a word in a given register is replace by the letter-by-letter encoding, followed by an "end of register" marker ##.   For example, if R1 has 1#1, then we encode this by 111#11##.  

We want to translate programs into ones which work on the encoded words, and we want to be sure that the translated programs have the feature that we are looking for. The basic idea in the programs below is that we can grab the symbols in R1 in a "two-by-two" fashion, and since we have an "end-of-register" marker ##, we will never do a case statement on an empty register.

For simplicity, we are going to work only with programs that are tidy and which only use R1, and which are then run on all empty registers.  This is what we need for the later use of this encoding, and these conditions could be relaxed a bit.


In [41]:
two_by_two_addone_idea = [ # this is sanity code for 1#, run on encoded R1
    ['top', 'cases', 1, 'empty', 'found_a_one', 'found_a_sharp'],
    ['empty','1###'],
    ['found_a_one', 'cases', 1, 'empty', 'one_one', 'one_sharp'],
    ['found_a_sharp', 'cases', 1, 'empty', 'empty', 'sharp_sharp'],
    ['one_one', '1#1#'],
    ['goto', 'top'],
    ['one_sharp', '1#1##'],
    ['goto', 'top'],
    ['sharp_sharp', '1#1#1##1##'], ## note that the 1#1# adds 11 before the end
    ['goto', 'end']
]

two_by_two_addone = sanity(two_by_two_addone_idea)

two_by_two_addsharp_idea = [ # this is sanity code for 1##, again run on encoded R1
    ['top', 'cases', 1, 'empty', 'found_a_one', 'found_a_sharp'],
    ['empty','1###'],
    ['found_a_one', 'cases', 1, 'empty', 'one_one', 'one_sharp'],
    ['found_a_sharp', 'cases', 1, 'empty', 'empty', 'sharp_sharp'],
    ['one_one', '1#1#'],
    ['goto', 'top'],
    ['one_sharp', '1#1##'],
    ['goto', 'top'],
    ['sharp_sharp', '1#1##1##1##'], ## note that the 1#1## adds 11 before the end
    ['goto', 'end']
]

two_by_two_addone = sanity(two_by_two_addone_idea)

two_by_two_addsharp = sanity(two_by_two_addsharp_idea)

In [15]:
onesharp(two_by_two_addsharp,['1###'])

'1#1###'

In [43]:
A = parse('11###1#')
B = [[x] for x in A]
C = [replace_add_instruction(c) for c in B]
print(B)
print(C)
print(sanity(C))
onesharp(sanity(C),['11##'])

[['11###'], ['1#']]
[['11###'], ['1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###']]
11###1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###


KeyboardInterrupt: ignored

In [211]:
def numbered(parsed): ## words on a parsed program
  n = len(parsed)
  x = [['a'+str(i+1),parsed[i]] for i in range(n)]
  return(x)

def resolve_transfer(program,instruction,index): # for use on an instruction in a numbered, parsed program  
  if instruction_type(instruction[1]) == 'forward':
     offset = len(instruction[1]) - 3
     #return([instruction[0], 'goto', str(index+offset)])
     if index+offset -1 == len(program):
       return([instruction[0], 'goto', 'end']) # note the special handling here
     else:
       return([instruction[0], 'goto', program[index+offset-1][0]])
  if instruction_type(instruction[1]) == 'backward':
     offset = len(instruction[1]) - 4
     return([instruction[0], 'goto', program[index-offset-1][0]])   
  else:
     return(instruction)   

def resolve_all_transfers(parsed): #for use on a parsed, numbered program 
  k = len(parsed)
  answer = [resolve_transfer(parsed,parsed[i-1],i) for i in range(1,k+1)]
  return(answer)

def replace_add_instruction(x):
  if x[1] == '1#':
    return([x[0],two_by_two_addone])
    ## this had been   return(two_by_two_addone])
    # but due to the fact that sanity expects 
    # input like ['1#1##'] rather than ['4', '1#1##'], the numbers are out
  elif  x[1] == '1##':   
    return([x[0],two_by_two_addsharp]) 
  else:
    return(x)

def replace_all_add_instructions(parsed): #for use on a parsed, numbered program 
  answer = [replace_add_instruction(i) for i in parsed]
  return(answer)

In [212]:
pzero = '11111###1#1#111####1#'
pone = parse(pzero)
print(pone)
print(len(pone))
p = numbered(pone)
print(p)
q = resolve_all_transfers(p)
print('q below')
print(q)
r =replace_all_add_instructions(q)
print('r')
print(r)
s = sanity(r)
print(s)
#parse_explain(s)
onesharp(s,['##'])

['11111###', '1#', '1#', '111####', '1#']
5
[['a1', '11111###'], ['a2', '1#'], ['a3', '1#'], ['a4', '111####'], ['a5', '1#']]
q below
[['a1', 'goto', 'end'], ['a2', '1#'], ['a3', '1#'], ['a4', 'goto', 'a1'], ['a5', '1#']]
r
[['a1', 'goto', 'end'], ['a2', '1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###'], ['a3', '1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###'], ['a4', 'goto', 'a1'], ['a5', '1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###']]
11111111111111111111111111111111111111111111111111111111111111111111111111###1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#

'##'

In [213]:
len(parse(s))

74

In [214]:
s

'11111111111111111111111111111111111111111111111111111111111111111111111111###1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###1111111111111111111111111111111111111111111111111####1#####111###111###111111###1###1#####11####111111###11111111###1#####111111####1111111####1111111###1#1#111111111111111####1#1##111111111111111111####1#1#1##1##1###'